### Notebook that walks through creation of database cluster using docker

Resources used:
- https://pymongo.readthedocs.io/en/stable/examples/high_availability.html
- https://www.mongodb.com/resources/products/compatibilities/deploying-a-mongodb-cluster-with-docker

Create a Docker Network

`docker network create mongoCluster`

Run docker instances

`docker-compose up -d`

In [ ]:
from pymongo import MongoClient

In [11]:
# Check whether replication is working
!docker exec -it mongo1 mongosh --eval "rs.status()"

]0;mongosh mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000MongoServerError: no replset config has been received


In [12]:
# Initialize the replica set - only once

client = MongoClient('localhost', 27017, directConnection=True)

config = {'_id': 'myReplicaSet', 'members': [
    {'_id': 0, 'host': 'mongo1:27017'},
    {'_id': 1, 'host': 'mongo2:27017'},
    {'_id': 2, 'host': 'mongo3:27017'}
]}
 
client.admin.command("replSetInitiate", config)

{'ok': 1.0,
 '$clusterTime': {'clusterTime': Timestamp(1746794335, 1),
  'signature': {'hash': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
   'keyId': 0}},
 'operationTime': Timestamp(1746794335, 1)}

In [13]:
# Check again

!docker exec -it mongo1 mongosh --eval "rs.status()"

]0;mongosh mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000{
  set: 'myReplicaSet',
  date: ISODate('2025-05-09T12:38:56.519Z'),
  myState: 2,
  term: Long('0'),
  syncSourceHost: '',
  syncSourceId: -1,
  heartbeatIntervalMillis: Long('2000'),
  majorityVoteCount: 2,
  writeMajorityCount: 2,
  votingMembersCount: 3,
  writableVotingMembersCount: 3,
  optimes: {
    lastCommittedOpTime: { ts: Timestamp({ t: 1746794335, i: 1 }), t: Long('-1') },
    lastCommittedWallTime: ISODate('2025-05-09T12:38:55.207Z'),
    readConcernMajorityOpTime: { ts: Timestamp({ t: 1746794335, i: 1 }), t: Long('-1') },
    appliedOpTime: { ts: Timestamp({ t: 1746794335, i: 1 }), t: Long('-1') },
    durableOpTime: { ts: Timestamp({ t: 1746794335, i: 1 }), t: Long('-1') },
    writtenOpTime: { ts: Timestamp({ t: 1746794335, i: 1 }), t: Long('-1') },
    lastAppliedWallTime: ISODate('2025-05-09T12:38:55.207Z'),
    lastDurableWallTime: ISODate('2025-05-09T12:38:55.207Z'),
    las

In [14]:
client = MongoClient('localhost', 27017, directConnection=True)

In [15]:
client.admin.command('ping')

{'ok': 1.0,
 '$clusterTime': {'clusterTime': Timestamp(1746794335, 1),
  'signature': {'hash': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
   'keyId': 0}},
 'operationTime': Timestamp(1746794335, 1)}

In [16]:
from pprint import pprint

# Health utils
def print_member_status(client):
    db = client.admin
    rs_status = db.command({'replSetGetStatus': 1})

    for m in rs_status['members']:
        print(m['name'], m['stateStr'])

print_member_status(client)

mongo1:27017 SECONDARY
mongo2:27017 SECONDARY
mongo3:27017 PRIMARY
